# Part 5: Graph to fDM

[![Notebook](https://shields.io/badge/notebook-access-green?logo=jupyter&style=for-the-badge)](https://github.com/cognitedata/neat/blob/main/docs/tutorial/notebooks/part-5-knowledge-graph-to-fdm-instances.ipynb)

* author: Nikola Vasiljevic
* date: 2023-10-22


In this Part 5 of tutorial series we will work with flexible Data Model(`fDM`) capabilities of Cognite Data Fusion. Unlike asset hierarchy, `fDM` provides full flexibility when come to modeling of domain. Instead of having generic assets, which one needs to enrich with metadata to make them more specific, with `fDM` one can model objects to ones will. In other words we are able to digitally model physical reality and store information, in the similar fashion like in case of our base `NeatGraph`.

`fDM` and modeled objects instances are stored in what is known as DMS (Data Model Storage), so often through out this tutorial you will see relations to `DMS`.

We strongly encourage reading Cognite material on data modeling:
- [About data modeling](https://docs.cognite.com/cdf/data_modeling/)
- [Developers docs on data modeling](https://developer.cognite.com/dm/)

We will start very light with `fDM`. For this purpose we will use [this rules](https://github.com/cognitedata/neat/blob/main/cognite/neat/examples/rules/sheet2cdf-transformation-rules.xlsx). For simplicity, we provide direct access through while doing import of necessary methods.


Also, for convenience store configuration of a Cognite client in `.env` file, with following structure:

```

TENANT_ID = ...
CLIENT_ID = ...
CLIENT_SECRET = ...
CDF_CLUSTER = ...
COGNITE_PROJECT = ...

```

This file will be loaded as config dictionary and used to configure the Cognite client.


Once you located necessary files, created `.env` file, load necessary libraries:

In [1]:
from dotenv import dotenv_values


from cognite.neat.rules.analysis import get_defined_classes
from cognite.neat.rules.examples import simple_example

from cognite.neat.rules import exporter, importer


from cognite.neat.graph.stores import NeatGraphStore

from cognite.neat.graph.loaders.rdf_to_dms import rdf2nodes_and_edges, upload_nodes, upload_edges


from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials


%reload_ext autoreload
%autoreload 2

In [3]:
config = dotenv_values("/Volumes/Secondary/repos/neat/playground/get.env")

SCOPES = [f"https://{config['CDF_CLUSTER']}.cognitedata.com/.default"]
TOKEN_URL = f"https://login.microsoftonline.com/{config['TENANT_ID']}/oauth2/v2.0/token"

credentials = OAuthClientCredentials(token_url=TOKEN_URL,
                                     client_id=config['CLIENT_ID'],
                                     client_secret=config['CLIENT_SECRET'],
                                     scopes=SCOPES)

client_config = ClientConfig(client_name="cognite",
                             base_url=f"https://{config['CDF_CLUSTER']}.cognitedata.com",
                             project=config['COGNITE_PROJECT'],
                             credentials=credentials,
                             max_workers=1,
                             timeout=5 * 60,)

client = CogniteClient(client_config)

Order of creating data model and its components is as following in `neat`:
1. Create `space` in CDF, where you will store `containers`, `views` and `data model`
2. Create `containers` in CDF, which will store data, where each container represents an object you defined in your data model
3. Create `views` on top of `containers`, which provide `view` into `container`
4. Create `data_model` as collection of `views`

`neat` will do this for you, all you need to do is to define data model through `Rules` and provide knowledge graph.

So let's start first with loading `Rules` and turning this one into `DataModel` class which contains methods for creating and uploading `data model` and its components:

In [28]:
rules = importer.ExcelImporter(simple_example).to_rules()

data_model = exporter.DMSExporter.from_rules(rules)

Let's inspect how many containers and views our data model contains:

In [29]:
dms_data_model.containers

,space,external_id,description,name,properties
0,playground,CountryGroup,A group of countries with the same dayahead sy...,CountryGroup,"{'name': {'type': {'list': False, 'collation':..."
1,playground,Country,A country with electric power markets.,Country,"{'name': {'type': {'list': False, 'collation':..."
2,playground,PriceArea,An area defined by the TSO to cotain the same ...,PriceArea,"{'name': {'type': {'list': False, 'collation':..."
3,playground,PriceAreaConnection,A virtual connection of two price areas,PriceAreaConnection,"{'name': {'type': {'list': False, 'collation':..."


In [30]:
dms_data_model.data_model.views

[<ViewApply(space='playground', external_id='CountryGroup', version='0_1') at 0x1328504d0>,
 <ViewApply(space='playground', external_id='Country', version='0_1') at 0x132eec210>,
 <ViewApply(space='playground', external_id='PriceArea', version='0_1') at 0x132d1de10>,
 <ViewApply(space='playground', external_id='PriceAreaConnection', version='0_1') at 0x132f1e510>]

As expected these containers/views correspond to `classes` we defined in `TransformationRules`:

In [31]:
get_defined_classes(rules)

{'Country', 'CountryGroup', 'PriceArea', 'PriceAreaConnection'}

Prior uploading data model and its components to Cognite Data Fusion, let's check under which `space` they will be stored:

In [32]:
dms_data_model.data_model.space

'playground'

To alter space under which data model and its components should be stored add or update field `cdfSpaceName` in the `TransformationRules` Excel file or change it directly here. For time being will stick to `playground` space, and upload model there:

In [33]:
client.data_modeling.containers.apply(dms_data_model.containers)

,space,external_id,description,name,used_for,properties,is_global,last_updated_time,created_time
0,playground,CountryGroup,A group of countries with the same dayahead sy...,CountryGroup,node,"{'name': {'type': {'list': False, 'collation':...",False,1699970060267,1699970060267
1,playground,Country,A country with electric power markets.,Country,node,"{'name': {'type': {'list': False, 'collation':...",False,1699970060267,1699970060267
2,playground,PriceArea,An area defined by the TSO to cotain the same ...,PriceArea,node,"{'name': {'type': {'list': False, 'collation':...",False,1699970060267,1699970060267
3,playground,PriceAreaConnection,A virtual connection of two price areas,PriceAreaConnection,node,"{'name': {'type': {'list': False, 'collation':...",False,1699970060267,1699970060267


In [34]:
client.data_modeling.data_models.apply(dms_data_model.data_model)

,value
space,playground
external_id,neat
description,This data model is solely used to demonstrate ...
name,NEAT Playground Data Model
version,0_1
views,"[{'space': 'playground', 'external_id': 'Count..."
is_global,False
last_updated_time,1699970064193
created_time,1699970064193


Loggin to CDF and go through Explore->Model your data->NEAT Playground Data Model, you should see something like this:
![](../../figs/DataModelCDF.PNG)

Let's now create instances of objects that we have defined, i.e. let's store data in `containes` we have created.
First we will create `NeatGraphStore` and then inject triples that define instances which are sotred in `Instances` sheet of `TransformationRules`:

In [35]:
graph_store = NeatGraphStore(prefixes=rules.prefixes,
                              namespace=rules.prefixes[rules.metadata.prefix])
graph_store.init_graph(base_prefix=rules.metadata.prefix)



for triple in exporter.TripleExporter.from_rules(rules).export():
    graph_store.graph.add(triple)

Now we will create `nodes` and `edges` that will be stored in containers:

In [25]:
from cognite.neat.rules.exporter._rules2dms import DataModel
data_model_neat = DataModel.from_rules(rules)

In [26]:
data_model_neat.remove_data_model(client)

In [36]:
nodes, edges, exceptions =  rdf2nodes_and_edges(graph_store, rules, "http://purl.org/cognite/neat#Nordics.Norway.NO1",add_class_prefix=True)

Let's look at two `nodes`, one being `Nordics.Norway.NO1` and another being `NO1.NO5`

In [37]:
nodes[2]

,value
instance_type,node
space,playground
external_id,PriceArea_Nordics.Norway.NO1
sources,"[{'properties': {'name': 'NO1', 'country': {'s..."


In [38]:
nodes[9]

,value
instance_type,node
space,playground
external_id,PriceAreaConnection_NO1.NO5
sources,"[{'properties': {'name': 'NO1 to NO5'}, 'sourc..."


These two nodes are connected via `edge`

In [39]:
edges[0]

,value
instance_type,edge
space,playground
external_id,PriceArea_Nordics.Norway.NO1-PriceAreaConnecti...
type,"{'space': 'playground', 'external_id': 'PriceA..."
start_node,"{'space': 'playground', 'external_id': 'PriceA..."
end_node,"{'space': 'playground', 'external_id': 'PriceA..."


As we now have `nodes` and `edges` we can upload them to `CDF`.
We start with uploading `nodes` followed by `edges`.

In [40]:
upload_nodes(client, nodes)
upload_edges(client, edges)

Let's now see results in Cognite Data Fusion:

![](../../figs/DataModelInstancesCDF.PNG)


If experiment features in Cognite Data Fusion are enable, one can see also visually graph: 

![](../../figs/DataModelInstancesVizCDF.PNG)